## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
"""
    DEEP RESEARCH

    can build it like a sidekick for yourself or to help with business research.
"""
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern. Also student Christian W. pointed out that OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [3]:
"""
    OPEN AI have a set of hosted tools that the public can be use to enrich their own agents

    You do not need to build MCPs for this.

    The WebSearchTool is not that cheap - 2.5cents per call, this can blow up.
"""
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    # This agent is required to run the tool, it does not have discretion over this.
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
message = "Latest AI Agent frameworks in 2026"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2026, several AI agent frameworks have emerged, each offering unique features:

- **OpenAI Agents SDK**: Provides a cohesive and rapid development environment for building AI agents, particularly suited for OpenAI's models. ([agentops.nu](https://agentops.nu/ai-agent-frameworks-2026/?utm_source=openai))

- **LangGraph**: Offers explicit control over agent behavior and durable state management, allowing developers to fine-tune agent operations. ([agentops.nu](https://agentops.nu/ai-agent-frameworks-2026/?utm_source=openai))

- **Microsoft Agent Framework**: An enterprise-grade solution integrating Semantic Kernel and AutoGen, delivering a unified toolkit for developing intelligent agents in corporate settings. ([linkedin.com](https://www.linkedin.com/pulse/5-ai-frameworks-dominate-software-architecture-2026-soundarapandian-logmf?utm_source=openai))

- **Orchestral**: A lightweight Python framework that provides a unified, type-safe interface for building LLM agents across major providers, simplifying tool integration and enhancing portability. ([arxiv.org](https://arxiv.org/abs/2601.02577?utm_source=openai))

- **MegaFlow**: A large-scale distributed orchestration system designed to efficiently manage complex agent-environment interactions, supporting large-scale training and evaluation of agents. ([arxiv.org](https://arxiv.org/abs/2601.07526?utm_source=openai))

- **AgentForge**: A modular framework for constructing autonomous agents driven by large language models, emphasizing composable skill abstractions and a unified LLM backend interface. ([arxiv.org](https://arxiv.org/abs/2601.13383?utm_source=openai))

- **Agent Lightning**: Enables reinforcement learning-based training of AI agents, decoupling agent execution from training processes and supporting complex interaction logic. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

These frameworks reflect the industry's focus on enhancing AI agent capabilities, scalability, and integration across various applications. 

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [ ]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    # Tell it how/why to populate these fields.
    # Asking for reasons result in more reasoning and better outputs.
    # Strange side effect of next predictive tokens - coherent output!
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

"""
    Come up with a list of searches that should be done for the input prompt!
    using WebSearchPlan sructured output.
"""
planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [6]:

message = "Latest AI Agent frameworks in 2026"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find current frameworks specific to AI agents being developed or used in 2026.', query='AI agent frameworks 2026'), WebSearchItem(reason='To explore leading companies and organizations that are pioneering new AI agent technologies in 2026.', query='top AI agent technologies 2026'), WebSearchItem(reason='To gather insights on trends and advancements in AI agent frameworks, including features and applications in 2026.', query='trends in AI agent development 2026')]


In [9]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com") # Change this to your verified email
    to_email = To("test@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return "success"

In [8]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x102982a20>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [10]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [11]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [12]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

# Search actually searches using the search agent using classin Runner.run
# we tell it the reason as well as the query - better context = better results.
async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

# Return aggregated searches based off the search_plan.
async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    # Build our list of tasks out which actually just perform the search() function.
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    # Use asyncio.gather to run tasks in parralel, this is true coroutine programming here.
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

### The next 2 functions write a report and email it

In [13]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    #result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent", report)
    return report

### Showtime!

In [ ]:
query ="Latest AI Agent frameworks in 2025"

"""
    Pretty minimal code for acheiveing deep research, a lot of potential here.
"""
with trace("Research trace"):
    print('Stsrting research')
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    print('Yay!')




Stsrting research
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Yay!


In [15]:
result

RunResult(input='Latest AI Agent frameworks in 2026', new_items=[MessageOutputItem(agent=Agent(name='PlannerAgent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer the query. Output 3 terms to query for.', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, prompt_cache_retention=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=<class '__main__.WebSearchPlan'>, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(i

### As always, take a look at the trace

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress, and a request</h2>
            <span style="color:#00cc00;">You've reached an important moment with the course; you've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/><br/>Something I should ask you -- my editor would smack me if I didn't mention this. If you're able to rate the course on Udemy, I'd be seriously grateful: it's the most important way that Udemy decides whether to show the course to others and it makes a massive difference.<br/><br/>And another reminder to <a href="https://www.linkedin.com/in/eddonner/">connect with me on LinkedIn</a> if you wish! If you wanted to post about your progress on the course, please tag me and I'll weigh in to increase your exposure.
            </span>
        </td>
    </tr>